In [2]:
from configparser import ConfigParser
import psycopg2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import recall_score, precision_score
from sklearn.preprocessing import MinMaxScaler

from collections import Counter


In [3]:
def config(filename='psql_sample.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

In [4]:
cfg = config()

In [5]:
try: 
    print ("Here's an attempt to connect to the database")
    conn = psycopg2.connect(**cfg)
    cursor = conn.cursor()
    print ("Look's like it was a success")
    
except (Exception, psycopg2.DatabaseError) as error: 
    print(error)

Here's an attempt to connect to the database
Look's like it was a success


In [6]:
try:
    cursor.execute("SELECT w.daily_high_tempreture, ph.city, p.gender, p.age_group, d.month, f.resolved, f.fatal FROM weather_dimension AS w, phu_location_dimension as ph, patient_dimension as p, date_dimension as d, fact_table as f WHERE w.weather_surrogate_key=f.weather_key AND ph.location_surrogate_key = f.location_key AND p.patient_surrogate_key=f.patient_key AND d.date_surrogate_key= f.onset_date_key"
                  )
    
    result_list = cursor.fetchall()
    
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

In [7]:
cursor.close()
conn.close()

In [8]:
result_df=pd.DataFrame(result_list, columns=["Temperature","City", "Gender", "Age Group", "Month", "Resolved","Fatal"])

In [9]:
result_df.head()
y= result_df["Month"]

In [10]:
new_result_df = pd.get_dummies(result_df, prefix=["City", "Gender", "Age Group"])
X = new_result_df
new_result_df.head()


,Temperature,Month,Resolved,Fatal,City_Mississauga,City_Newmarket,City_Oakville,City_Ottawa,City_Toronto,City_Whitby,...,Age Group_20s,Age Group_30s,Age Group_40s,Age Group_50s,Age Group_60s,Age Group_70s,Age Group_80s,Age Group_90+,Age Group_<20,Age Group_UNKNOWN
0,20.9,9,True,False,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,22.5,9,True,False,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,19.4,9,True,False,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,19.3,9,True,False,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,22.1,9,True,False,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [18]:
normalized_df = new_result_df.copy()
normalized_df["Temperature"]= MinMaxScaler().fit_transform(np.array(normalized_df["Temperature"]).reshape(-1,1))
normalized_df["Resolved"] = normalized_df["Resolved"].astype(int)
normalized_df["Fatal"] = normalized_df["Fatal"].astype(int)

normalized_df.to_csv('dataframe.csv')